# Notebook 00: Baselines and Limitations

## Rigorous Evaluation of Classical ML for Nuclear Data

**Learning Objective:** Understand *why* classical machine learning fails for nuclear data evaluation through a rigorous, research-grade comparison using optimal hyperparameters.

**Methodology:**
- Use **Bayesian hyperparameter optimization** (hyperopt) to find optimal configurations
- Train on **full EXFOR database** (~3.6M measurements) for maximum data availability
- Evaluate on **data-rich** (U-235) and **data-sparse** (Cl-35) isotopes
- Compare against physics requirements (smoothness, extrapolation, constraints)

**Focus Isotopes:**
- **U-235 Fission** (data-rich, well-understood): Critical for nuclear reactors
- **Cl-35 (n,p)** (data-sparse, research interest): Important for astrophysics and medical applications

### The Problem

Nuclear cross sections σ(E) are smooth, continuous functions of energy. They exhibit:
- **Resonance peaks**: Sharp but smooth features (especially visible in U-235)
- **Threshold behavior**: σ(E) = 0 for E < E_threshold, then rises smoothly
- **Physical constraints**: Conservation laws, unitarity, causality

### Why This Matters

A reactor calculation uses millions of cross-section evaluations. If predictions are:
- **Jagged** → Unphysical neutron transport
- **Discontinuous** → Numerical instabilities
- **Wrong at key energies** → Incorrect k_eff (criticality)

This is the **Validation Paradox**: Low MSE ≠ Safe Reactor!

**Additional Challenge:** How do models perform when data is sparse (like Cl-35)?

---

## Part 1: Optimized Decision Trees

We'll use **hyperparameter optimization** to give Decision Trees their best chance. This eliminates the strawman of "intentionally bad" parameters and provides a fair, rigorous comparison.

In [ ]:
import sys
sys.path.append('..')

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path

from nucml_next.data import NucmlDataset
from nucml_next.baselines import XGBoostEvaluator, DecisionTreeEvaluator

# Set plotting style
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")
%matplotlib inline

# Verify EXFOR data exists
exfor_path = Path('../data/exfor_processed.parquet')
if not exfor_path.exists():
    raise FileNotFoundError(
        f"EXFOR data not found at {exfor_path}\n"
        "Please run: python scripts/ingest_exfor.py --x4-db data/x4sqlite1.db --output data/exfor_processed.parquet"
    )

print("✓ Imports successful")
print("✓ EXFOR data found")
print("Welcome to NUCML-Next: Understanding ML Limitations with Real Nuclear Data")

### Configuration: Feature Tiers and Transformations

Before loading data, configure which features and transformations to use.

In [ ]:
# ============================================================================
# USER CONFIGURATION: Feature Tiers and Transformations
# ============================================================================
# Change these settings in ONE place instead of scattered throughout the notebook

from nucml_next.data.selection import TransformationConfig

# ============================================================================
# FEATURE TIER SELECTION
# ============================================================================
# Choose which AME2020/NUBASE2020 enrichment tiers to include
#
# Tier A (13 features) - ALWAYS INCLUDED:
#   - Z, A, N, Energy (nuclear coordinates)
#   - 9-feature Numerical Particle Vector:
#     out_n, out_p, out_a, out_g, out_f, out_t, out_h, out_d, is_met
#
# Tier B (+2 features) - Geometric:
#   + R_fm (nuclear radius)
#   + kR (dimensionless interaction parameter)
#
# Tier C (+7 features) - Energetics: ✅ RECOMMENDED FOR BASELINES
#   + Mass_Excess_MeV (mass excess)
#   + Binding_Energy_MeV (total binding energy)
#   + Binding_Per_Nucleon_MeV (B/A)
#   + S_1n_MeV, S_2n_MeV (neutron separation energies)
#   + S_1p_MeV, S_2p_MeV (proton separation energies)
#
# Tier D (+8 features) - Topological:
#   + Spin, Parity (nuclear structure)
#   + Valence_N, Valence_P (distance to magic numbers)
#   + P_Factor (pairing: even-even/odd-odd)
#   + Shell_Closure_N, Shell_Closure_P
#
# Tier E (+8 features) - Complete Q-values:
#   + Q_alpha_MeV, Q_2beta_minus_MeV, Q_ep_MeV, etc.
#   + All 8 reaction Q-values from AME2020

SELECTED_TIERS = ['A', 'C']  # 🎯 Change tiers HERE (only place to modify)

print(f"📊 Selected Feature Tiers: {SELECTED_TIERS}")
print(f"   Features: Tier A (core + particle vector) + Tier C (energetics)")
print()

# ============================================================================
# TRANSFORMATION CONFIGURATION
# ============================================================================
# Configure log-scaling and standardization for ML training
#
# This controls how features and targets are transformed before training.
# All transformations are reversible for predictions.

TRANSFORMATION_CONFIG = TransformationConfig(
    # Target (cross-section) transformations
    log_target=True,              # Enable log₁₀ transform for cross-sections
                                  # Stabilizes gradients and handles wide σ range (mb to kb)
    
    target_epsilon=1e-10,         # Epsilon for log(σ + ε) to prevent log(0)
                                  # Increase if you have very small cross-sections
    
    log_base=10,                  # Logarithm base: 10 | 'e' | 2
                                  # Base-10 is standard in nuclear physics
    
    # Energy transformations
    log_energy=True,              # Enable log₁₀ transform for energies
                                  # Handles wide energy range (eV to MeV)
    
    energy_log_base=10,           # Energy log base: 10 | 'e' | 2
    
    # Feature standardization
    scaler_type='standard',       # Feature scaling method:
                                  # 'standard' = Z-score normalization (X-μ)/σ [DEFAULT]
                                  # 'minmax'   = Min-max scaling to [0,1]
                                  # 'robust'   = Robust scaling using median and IQR
                                  # 'none'     = No scaling (use raw features)
    
    scale_features=None           # Columns to scale. None = auto-detect numeric columns
                                  # Example: ['Z', 'A', 'N', 'Mass_Excess_MeV']
)

print("🔧 Transformation Configuration:")
print(TRANSFORMATION_CONFIG)
print()
print("💡 To change settings, modify SELECTED_TIERS and TRANSFORMATION_CONFIG above")
print("=" * 80)

### Step 1.1: Load Real EXFOR Data (Tabular View)

We'll train on the **full EXFOR database** (all isotopes), then evaluate on U-235 and Cl-35. This is the correct ML approach:
- **Training**: Learn general nuclear physics patterns from ALL available data
- **Evaluation**: Test predictions on specific target isotopes (U-235, Cl-35)

This demonstrates true transfer learning and generalization, not just memorization!

In [ ]:
# CRITICAL FIX: Use physics-aware DataSelection for scientifically defensible filtering
# This demonstrates proper ML workflow with predicate pushdown for efficiency

from nucml_next.data import DataSelection

# Strategy: Train on neutron-induced reactions at reactor energies (scientifically defensible default)
# This avoids training on:
# - Non-reactor energies (too high/low for criticality calculations)
# - Non-neutron projectiles (we're focused on reactor physics)
# 
# NOTE: We now INCLUDE all MT codes (including 0, 1, and >=9000) in 'all_physical' mode

print("Creating physics-aware data selection...")
print("=" * 80)

# Training selection: Reactor physics, neutrons, all physical reactions
training_selection = DataSelection(
    # ============================================================================
    # PROJECTILE SELECTION
    # ============================================================================
    projectile='neutron',          # Options: 'neutron' | 'all'
                                   # 'neutron' = Only neutron-induced reactions (reactor physics)
                                   # 'all' = All projectiles (n, p, d, α, γ, etc.)
    
    # ============================================================================
    # ENERGY RANGE (eV)
    # ============================================================================
    energy_min=1e-5,               # Minimum energy in eV (1e-5 = 0.01 eV, thermal neutrons)
    energy_max=2e7,                # Maximum energy in eV (2e7 = 20 MeV, reactor physics upper bound)
                                   # Common ranges:
                                   #   - Thermal: 1e-5 to 1 eV
                                   #   - Resonance: 1 to 1e4 eV
                                   #   - Fast: 1e4 to 2e7 eV (20 MeV)
                                   #   - High energy: up to 1e9 eV (1 GeV)
    
    # ============================================================================
    # REACTION (MT) MODE SELECTION
    # ============================================================================
    mt_mode='all_physical',        # Options:
                                   # 'reactor_core'   → Essential for reactor modeling
                                   #                    (MT 2, 4, 16, 18, 102, 103, 107)
                                   #                    [elastic, inelastic, (n,2n), fission,
                                   #                     capture, (n,p), (n,α)]
                                   #
                                   # 'threshold_only' → Reactions with energy thresholds
                                   #                    (MT 16, 17, 103, 104, 105, 106, 107)
                                   #                    [(n,2n), (n,3n), (n,p), (n,d), (n,t),
                                   #                     (n,³He), (n,α)]
                                   #
                                   # 'fission_details'→ Fission breakdown channels
                                   #                    (MT 18, 19, 20, 21, 38)
                                   #                    [total fission, 1st chance, 2nd chance,
                                   #                     3rd chance, 4th chance]
                                   #
                                   # 'all_physical'   → All MT codes including bookkeeping
                                   #                    (MT 0, 1, and >=9000 now INCLUDED)
                                   #
                                   # 'custom'         → Use custom_mt_codes list (see below)
    
    custom_mt_codes=None,          # Used only when mt_mode='custom'
                                   # Example: [2, 18, 102]  # Elastic, fission, capture
                                   # Example: [16, 17, 18]  # (n,2n), (n,3n), fission
                                   # Example: list(range(50, 92))  # MT 50-91 (inelastic levels)
    
    # ============================================================================
    # EXCLUSION RULES
    # ============================================================================
    exclude_bookkeeping=False,     # Set to False to INCLUDE MT 0, 1, and MT >= 9000
                                   # MT 0 = Undefined
                                   # MT 1 = Total cross-section (sum of others)
                                   # MT >= 9000 = Lumped reaction covariances
                                   # Now including these for comprehensive analysis
    
    # ============================================================================
    # DATA VALIDITY
    # ============================================================================
    drop_invalid=True,             # Drop NaN or non-positive cross-sections
                                   # Essential for log-transform: log(σ) requires σ > 0
                                   # Prevents training instabilities
    
    # ============================================================================
    # EVALUATION CONTROLS (Holdout for Extrapolation Testing)
    # ============================================================================
    holdout_isotopes=None,         # List of (Z, A) tuples to exclude from training
                                   # None = Use all data (default for training)
                                   # Example: [(92, 235)]           # Hold out U-235 only
                                   # Example: [(92, 235), (17, 35)] # Hold out U-235 and Cl-35
                                   # Example: [(94, 239), (94, 240), (94, 241)]  # Pu isotopes
                                   # Use this to measure TRUE extrapolation capability!
    
    # ============================================================================
    # AME2020/NUBASE2020 ENRICHMENT TIER SELECTION
    # ============================================================================
    tiers=SELECTED_TIERS,          # 🎯 Using centralized tier configuration
    transformation_config=TRANSFORMATION_CONFIG  # Using centralized transformation config
)

print("Training Selection:")
print(training_selection)
print()

# Load FULL dataset for training with physics-aware filtering
# CRITICAL: Predicate pushdown filters at PyArrow fragment level (90% I/O reduction!)
print("=" * 80)
print("Loading training dataset with predicate pushdown...")
print("=" * 80)
dataset_full = NucmlDataset(
    data_path='../data/exfor_processed.parquet',
    mode='tabular',
    selection=training_selection  # Physics-aware selection with predicate pushdown
)

# ============================================================================
# Apply manual energy filter (PyArrow energy filter not working correctly)
# ============================================================================
# PyArrow's predicate pushdown is not correctly applying energy filter.
# Despite specifying energy_max=2e7 (20 MeV), data up to 1 GeV is loaded.
#
# TEMPORARY FIX: Apply manual energy filter after loading

print("\n⚠️  Applying manual filters (PyArrow bug workarounds)...")
original_size = len(dataset_full.df)

# Filter: Energy range (should be 1e-5 to 2e7, but PyArrow loads up to 1e9)
dataset_full.df = dataset_full.df[
    (dataset_full.df['Energy'] >= training_selection.energy_min) &
    (dataset_full.df['Energy'] <= training_selection.energy_max)
].copy()
after_energy = len(dataset_full.df)

print(f"  Energy filter:     {original_size:,} → {after_energy:,} ({after_energy/original_size*100:.1f}%)")
print(f"  Total filtered:    {original_size:,} → {after_energy:,} ({after_energy/original_size*100:.1f}%)")

# ============================================================================
# Project to tabular format with TIER-BASED features (particle vector)
# ============================================================================
print("\n" + "=" * 80)
print("Projecting to tabular format (tier mode with particle vector)...")
print("=" * 80)

    tiers=SELECTED_TIERS,          # 🎯 Using centralized tier configuration
print(f"\n✓ Training dataset: {df_tier.shape}")
print(f"  Energy range: {df_tier['Energy'].min():.2e} to {df_tier['Energy'].max():.2e} eV")
print(f"\n📋 Features Available:")
print(f"  Tier A features: Z, A, N, Energy + 9-feature particle vector")
print(f"    Particle vector: out_n, out_p, out_a, out_g, out_f, out_t, out_h, out_d, is_met")
print(f"  Tier C features: Mass excess, binding, separation energies")
print(f"  Total features: {len(df_tier.columns)}")
print(f"\nFeature names:")
print(df_tier.columns.tolist())

# Show isotope distribution in training data
print("\n📊 Training Data Distribution (Top 10 Isotopes):")
isotope_counts = dataset_full.df.groupby(['Z', 'A']).size().sort_values(ascending=False).head(10)
for (z, a), count in isotope_counts.items():
    # Simple element lookup (extend as needed)
    element_map = {92: 'U', 17: 'Cl', 94: 'Pu', 26: 'Fe', 8: 'O', 1: 'H',
                   82: 'Pb', 6: 'C', 13: 'Al', 7: 'N', 11: 'Na', 79: 'Au'}
    elem = element_map.get(z, f'Z{z}')
    print(f"  {elem}-{a:3d}: {count:>8,} measurements")

print(f"\n✓ Total isotopes: {dataset_full.df.groupby(['Z', 'A']).ngroups} unique Z/A combinations")
print(f"✓ Total reaction types: {dataset_full.df['MT'].nunique()} unique MT codes")
print(f"✓ Total measurements: {len(dataset_full.df):,}")

# Show MT distribution
print("\n📊 Top 10 Reaction Types (MT codes):")
mt_counts = dataset_full.df['MT'].value_counts().head(10)
mt_names = {18: 'Fission', 102: '(n,γ) Capture', 103: '(n,p)', 2: 'Elastic',
            16: '(n,2n)', 17: '(n,3n)', 4: 'Inelastic', 107: '(n,α)',
            0: 'Undefined', 1: 'Total XS'}
for mt, count in mt_counts.items():
    name = mt_names.get(mt, f'MT-{mt}')
    print(f"  MT {mt:3d} {name:15s}: {count:>8,} measurements")

print(f"\n✓ Training on neutron-induced reactions (reactor energies 0.01 eV - 20 MeV)")
print(f"✓ Including ALL MT codes (0, 1, and >=9000 now included)")
print(f"✓ Using Tier A + C features with particle vector transformation")

# Now load evaluation targets (U-235 and Cl-35) using legacy filters for specific selection
print("\n" + "="*70)
print("Loading evaluation targets (U-235 and Cl-35)...")
print(f"NOTE: Using same energy range as training: {training_selection.energy_min:.2e} to {training_selection.energy_max:.2e} eV")
print("="*70)

# Create evaluation selection with same energy limits but specific isotopes
eval_selection = DataSelection(
    projectile='neutron',
    energy_min=training_selection.energy_min,  # SAME as training
    energy_max=training_selection.energy_max,  # SAME as training  
    mt_mode='custom',
    custom_mt_codes=[18, 102, 103],  # Fission, capture, (n,p) - for visualization
    exclude_bookkeeping=False,  # Include all MT codes
    drop_invalid=True,
    tiers=SELECTED_TIERS,          # 🎯 Using centralized tier configuration
)

dataset_eval = NucmlDataset(
    data_path='../data/exfor_processed.parquet',
    mode='tabular',
    selection=eval_selection
)

# Apply same manual energy filter (workaround for PyArrow bugs)
dataset_eval.df = dataset_eval.df[
    (dataset_eval.df['Energy'] >= eval_selection.energy_min) &
    (dataset_eval.df['Energy'] <= eval_selection.energy_max)
].copy()

# Filter to U-235 and Cl-35 only
dataset_eval.df = dataset_eval.df[
    ((dataset_eval.df['Z'] == 92) & (dataset_eval.df['A'] == 235)) |
    ((dataset_eval.df['Z'] == 17) & (dataset_eval.df['A'] == 35))
].copy()

print(f"✓ Evaluation dataset: {len(dataset_eval.df)} measurements")
print(f"  Energy range: {dataset_eval.df['Energy'].min():.2e} to {dataset_eval.df['Energy'].max():.2e} eV")
print("\n📊 Evaluation Isotopes:")
for (z, a), group in dataset_eval.df.groupby(['Z', 'A']):
    isotope = f"{'U' if z==92 else 'Cl'}-{a}"
    e_min = group['Energy'].min()
    e_max = group['Energy'].max()
    print(f"  {isotope:8s}: {len(group):>6,} measurements (Energy: {e_min:.2e} to {e_max:.2e} eV)")
print("="*70)

df_tier.head()

**Notice:** The naive approach treats reactions as independent categories (MT_2, MT_18, etc.).

**Problem:** This ignores physics! (n,2n) and (n,3n) are related - they differ by one neutron.

But tree-based models don't know this. To them, MT=16 and MT=17 are just labels.

### Step 1.2: Optimize and Train Decision Tree

Instead of artificially limiting the model, we'll use **Bayesian hyperparameter optimization** to find the optimal Decision Tree configuration. This is a research-grade approach that gives classical ML its best chance to succeed.

**Research Question**: Even with optimal hyperparameters, can Decision Trees produce smooth, physics-compliant cross-section predictions?

In [ ]:
# ============================================================================
# HYPERPARAMETER OPTIMIZATION FOR DECISION TREE
# ============================================================================
# Use Bayesian optimization (hyperopt) to find optimal hyperparameters
# This ensures we're evaluating Decision Trees at their BEST, not strawman configs

print("="*80)
print("STEP 1: HYPERPARAMETER OPTIMIZATION")
print("="*80)
print("Finding optimal Decision Tree configuration using Bayesian optimization...")
print("This may take 5-10 minutes on large datasets.\n")

# Initialize temporary model for optimization
dt_optimizer = DecisionTreeEvaluator()

# Run hyperparameter optimization (50 evaluations for speed, increase for production)
opt_result_dt = dt_optimizer.optimize_hyperparameters(
    df_tier,
    max_evals=50,          # Increase to 100+ for production
    cv_folds=3,            # 3-fold cross-validation
    verbose=True
)

print("\n" + "="*80)
print("STEP 2: TRAINING FINAL MODEL WITH OPTIMAL HYPERPARAMETERS")
print("="*80)

# Create model with optimal hyperparameters
dt_model = DecisionTreeEvaluator(**opt_result_dt['best_params'])

# Train on full dataset
dt_metrics = dt_model.train(df_tier)

print("\n" + "="*60)
print("Optimized Decision Tree Performance:")
print("="*60)
for key, value in dt_metrics.items():
    print(f"  {key:20s}: {value}")

print("\n💡 Note: This is an OPTIMIZED Decision Tree, not artificially constrained.")
print("   The staircase effect is inherent to the algorithm, not poor hyperparameters!")

### Step 1.3: The Failure Mode - Visualize the Staircase Effect

Let's predict cross sections for both isotopes and see what happens...

**U-235**: Rich data → Can the model capture resonances?  
**Cl-35**: Sparse data → Can the model interpolate gaps?

In [ ]:
# ============================================================================
# HELPER FUNCTIONS for Clean Cross-Section Plotting
# ============================================================================

def _clean_xy(df, col_x='Energy', col_y='CrossSection', emin=None, emax=None):
    """
    Clean and prepare data for log-log plotting.
    
    - Drops NaN values in Energy and CrossSection
    - Removes non-positive Energy and CrossSection (required for log scale)
    - Filters by energy range if specified
    - Sorts by Energy to avoid line-crossing artifacts
    
    Args:
        df: DataFrame with Energy and CrossSection columns
        col_x: Name of energy column
        col_y: Name of cross-section column
        emin: Minimum energy (eV), optional
        emax: Maximum energy (eV), optional
    
    Returns:
        Cleaned DataFrame sorted by Energy
    """
    df_clean = df.copy()
    
    # Drop NaN values
    df_clean = df_clean.dropna(subset=[col_x, col_y])
    
    # Remove non-positive values (required for log scale)
    df_clean = df_clean[(df_clean[col_x] > 0) & (df_clean[col_y] > 0)]
    
    # Apply energy range filter if specified
    if emin is not None:
        df_clean = df_clean[df_clean[col_x] >= emin]
    if emax is not None:
        df_clean = df_clean[df_clean[col_x] <= emax]
    
    # CRITICAL: Sort by Energy to avoid line-crossing artifacts
    df_clean = df_clean.sort_values(by=col_x).reset_index(drop=True)
    
    return df_clean


def _clip_positive(arr, floor=1e-30):
    """
    Clip array values to minimum positive floor.
    
    Prevents log(0) errors by ensuring all values are at least 'floor'.
    Useful for prediction arrays that may contain zeros or negatives.
    
    Args:
        arr: Numpy array or list
        floor: Minimum positive value (default: 1e-30)
    
    Returns:
        Numpy array with values >= floor
    """
    arr = np.asarray(arr)
    return np.clip(arr, floor, None)


# ============================================================================
# FIXED ENERGY RANGE FOR PLOTS
# ============================================================================
E_MIN_PLOT = 1e-4   # 10^-4 eV
E_MAX_PLOT = 1e7    # 10^7 eV

# ============================================================================
# CREATE COMPARATIVE VISUALIZATION: U-235 (data-rich) vs Cl-35 (data-sparse)
# ============================================================================

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(16, 7))

# ============================================================================
# LEFT PANEL: U-235 Fission (Data-Rich)
# ============================================================================

Z_u, A_u, mt_u = 92, 235, 18  # U-235 Fission

# Extract U-235 fission data
mask_u = (dataset_eval.df['Z'] == Z_u) & (dataset_eval.df['A'] == A_u) & (dataset_eval.df['MT'] == mt_u)
df_u = _clean_xy(dataset_eval.df[mask_u], emin=E_MIN_PLOT, emax=E_MAX_PLOT)

# Get Decision Tree predictions
energies_dt_u, predictions_dt_u = dt_model.predict_resonance_region(
    Z_u, A_u, mt_u, (E_MIN_PLOT, E_MAX_PLOT), num_points=500
)
predictions_dt_u = _clip_positive(predictions_dt_u)

# Plot ground truth (blue scatter, behind)
ax1.scatter(df_u['Energy'], df_u['CrossSection'], 
            s=30, c='tab:blue', marker='o', 
            label=f'Ground Truth ({len(df_u)} EXFOR pts)', 
            alpha=0.5, zorder=1, edgecolors='none')

# Plot Decision Tree prediction (red line, on top)
ax1.plot(energies_dt_u, predictions_dt_u, 
         'tab:red', linewidth=2.5, 
         label='Decision Tree (Staircase)', 
         alpha=0.9, zorder=3)

# Configure axes
ax1.set_xscale('log')
ax1.set_yscale('log')
ax1.set_xlim(E_MIN_PLOT, E_MAX_PLOT)
ax1.set_xlabel('Energy (eV)', fontsize=12, fontweight='bold')
ax1.set_ylabel('Cross Section (barns)', fontsize=12, fontweight='bold')
ax1.set_title('U-235 Fission (DATA-RICH): Staircase Effect\n' + 
              f'{len(df_u)} EXFOR measurements in range\n' +
              '(Model trained on full EXFOR database)',
              fontsize=13, fontweight='bold')
ax1.legend(fontsize=11, loc='best')
ax1.grid(True, alpha=0.3, which='both')

# Annotate the problem
if len(predictions_dt_u) > 250:
    mid_idx = 250
    ax1.annotate('Unphysical steps!\n(Real resonances are smooth)',
                 xy=(energies_dt_u[mid_idx], predictions_dt_u[mid_idx]), 
                 xytext=(energies_dt_u[mid_idx] * 2, predictions_dt_u[mid_idx] * 3),
                 arrowprops=dict(arrowstyle='->', color='tab:red', lw=2),
                 fontsize=10, color='tab:red', fontweight='bold',
                 bbox=dict(boxstyle='round', facecolor='yellow', alpha=0.7))

# ============================================================================
# RIGHT PANEL: Cl-35 (n,p) (Data-Sparse)
# ============================================================================

Z_cl, A_cl, mt_cl = 17, 35, 103  # Cl-35 (n,p)

# Extract Cl-35 (n,p) data
mask_cl = (dataset_eval.df['Z'] == Z_cl) & (dataset_eval.df['A'] == A_cl) & (dataset_eval.df['MT'] == mt_cl)
df_cl = _clean_xy(dataset_eval.df[mask_cl], emin=E_MIN_PLOT, emax=E_MAX_PLOT)

# Get Decision Tree predictions
energies_dt_cl, predictions_dt_cl = dt_model.predict_resonance_region(
    Z_cl, A_cl, mt_cl, (E_MIN_PLOT, E_MAX_PLOT), num_points=500
)
predictions_dt_cl = _clip_positive(predictions_dt_cl)

# Plot ground truth (blue scatter, behind)
ax2.scatter(df_cl['Energy'], df_cl['CrossSection'], 
            s=80, c='tab:blue', marker='o', 
            label=f'Ground Truth ({len(df_cl)} EXFOR pts)', 
            alpha=0.6, zorder=1, edgecolors='black', linewidths=1)

# Plot Decision Tree prediction (red line, on top)
ax2.plot(energies_dt_cl, predictions_dt_cl, 
         'tab:red', linewidth=2.5, 
         label='Decision Tree (Extrapolation)', 
         alpha=0.9, zorder=3)

# Configure axes
ax2.set_xscale('log')
ax2.set_yscale('log')
ax2.set_xlim(E_MIN_PLOT, E_MAX_PLOT)
ax2.set_xlabel('Energy (eV)', fontsize=12, fontweight='bold')
ax2.set_ylabel('Cross Section (barns)', fontsize=12, fontweight='bold')
ax2.set_title('Cl-35 (n,p) (DATA-SPARSE): Transfer Learning Test\n' + 
              f'Only {len(df_cl)} EXFOR measurements!\n' +
              '(Model learned from other isotopes)',
              fontsize=13, fontweight='bold')
ax2.legend(fontsize=11, loc='best')
ax2.grid(True, alpha=0.3, which='both')

# Annotate the challenge
if len(predictions_dt_cl) > 250:
    mid_idx = 250
    ax2.annotate('Can the model\ntransfer knowledge?',
                 xy=(energies_dt_cl[mid_idx], predictions_dt_cl[mid_idx]), 
                 xytext=(energies_dt_cl[mid_idx] * 1.5, predictions_dt_cl[mid_idx] * 0.5),
                 arrowprops=dict(arrowstyle='->', color='tab:red', lw=2),
                 fontsize=10, color='tab:red', fontweight='bold',
                 bbox=dict(boxstyle='round', facecolor='yellow', alpha=0.7))

plt.tight_layout()
plt.show()

# ============================================================================
# OBSERVATIONS
# ============================================================================

print("\n⚠️  OBSERVATIONS:")
print("="*80)
print(f"Fixed Plot Range: {E_MIN_PLOT:.2e} to {E_MAX_PLOT:.2e} eV")
print("Training Approach: Model trained on FULL EXFOR database (all isotopes)")
print("Evaluation: Testing predictions on U-235 and Cl-35")
print()
print("LEFT (U-235 - Data-Rich in training):")
print("  • Decision Tree creates JAGGED predictions even with lots of training data")
print("  • Staircase effect would cause numerical instabilities in reactor codes")
print("  • Blue scatter: EXFOR ground truth (cleaned and sorted)")
print("  • Red line: Decision Tree prediction showing discontinuities")
print()
print("RIGHT (Cl-35 - Data-Sparse, transfer learning):")
print("  • Model must transfer knowledge from other isotopes")
print("  • Large gaps between measurements → Predictions test generalization")
print("  • Blue scatter: Sparse EXFOR measurements")
print("  • Red line: Decision Tree extrapolation")
print("  • This is where physics-informed models REALLY shine!")
print("="*80)

### 🔴 Critical Insight #1: Piecewise Constant ≠ Physics

Decision trees partition feature space into rectangles:
```
if Energy < 10.5:
    if Energy < 5.2:
        return 150.0  # Constant!
    else:
        return 89.0   # Jump!
else:
    return 45.0
```

Real physics:
```
σ(E) = σ_0 * Γ / ((E - E_r)² + Γ²/4)  # Smooth Breit-Wigner!
```

**Important**: The staircase effect persists even with optimal hyperparameters. This is a fundamental architectural limitation, not a tuning problem!

---

## Part 2: Can XGBoost Save Us?

Let's try a more sophisticated ensemble method with optimal hyperparameters.

In [ ]:
# ============================================================================
# HYPERPARAMETER OPTIMIZATION FOR XGBOOST (NAIVE FEATURES)
# ============================================================================

print("="*80)
print("STEP 1: HYPERPARAMETER OPTIMIZATION - XGBoost (Naive Features)")
print("="*80)
print("Finding optimal XGBoost configuration using Bayesian optimization...")
print("This may take 10-15 minutes on large datasets.\n")

# Initialize temporary model for optimization
xgb_optimizer_naive = XGBoostEvaluator()

# Run hyperparameter optimization
opt_result_xgb_naive = xgb_optimizer_naive.optimize_hyperparameters(
    df_tier,
    max_evals=50,          # Increase to 100+ for production
    cv_folds=3,
    verbose=True
)

print("\n" + "="*80)
print("STEP 2: TRAINING FINAL MODEL WITH OPTIMAL HYPERPARAMETERS")
print("="*80)

# Create model with optimal hyperparameters
xgb_naive = XGBoostEvaluator(**opt_result_xgb_naive['best_params'])

# Train on full dataset
xgb_metrics_naive = xgb_naive.train(df_tier)

print("\n" + "="*60)
print("Optimized XGBoost Performance (Naive Features):")
print("="*60)
for key, value in xgb_metrics_naive.items():
    if value is not None:
        print(f"  {key:20s}: {value}")

print("\n💡 Note: These are OPTIMAL hyperparameters found via Bayesian optimization.")
print("   Any remaining issues are architectural, not due to poor tuning!")

In [ ]:
# ============================================================================
# FIXED ENERGY RANGE FOR COMPARISON PLOTS
# ============================================================================
E_MIN_PLOT = 1e-4   # 10^-4 eV
E_MAX_PLOT = 1e7    # 10^7 eV

# U-235 fission comparison
Z, A, mt_code = 92, 235, 18
energy_range = (E_MIN_PLOT, E_MAX_PLOT)

# Get XGBoost predictions
energies_xgb, predictions_xgb = xgb_naive.predict_resonance_region(
    Z, A, mt_code, energy_range, num_points=1000
)

# Get ground truth from evaluation dataset
mask = (dataset_eval.df['Z'] == Z) & (dataset_eval.df['A'] == A) & (dataset_eval.df['MT'] == mt_code)
df_truth = dataset_eval.df[mask].copy()
df_truth = df_truth[(df_truth['Energy'] >= energy_range[0]) & (df_truth['Energy'] <= energy_range[1])]

# Get Decision Tree predictions
energies_dt, predictions_dt = dt_model.predict_resonance_region(
    Z, A, mt_code, energy_range, num_points=1000
)

# Comparative plot
fig, ax = plt.subplots(figsize=(12, 6))

# Ground truth - BEHIND (zorder=1)
ax.plot(df_truth['Energy'], df_truth['CrossSection'], 
        'b-', linewidth=2.5, label='Ground Truth (EXFOR)', alpha=0.6, zorder=1)

# Decision Tree (stairs) - MIDDLE (zorder=2)
ax.plot(energies_dt, predictions_dt, 
        'r--', linewidth=2.0, label='Decision Tree (Staircase)', alpha=0.8, zorder=2)

# XGBoost (smoother but not smooth) - ON TOP (zorder=3)
ax.plot(energies_xgb, predictions_xgb, 
        'g-', linewidth=2.5, label='XGBoost (Better, but...)', alpha=0.9, zorder=3)

ax.set_xscale('log')
ax.set_yscale('log')
ax.set_xlim(E_MIN_PLOT, E_MAX_PLOT)
ax.set_xlabel('Energy (eV)', fontsize=12, fontweight='bold')
ax.set_ylabel('Cross Section (barns)', fontsize=12, fontweight='bold')
ax.set_title('XGBoost vs Decision Tree: Improvement but Still Not Physics-Compliant\nU-235 Fission (Model trained on full EXFOR)',
             fontsize=14, fontweight='bold')
ax.legend(fontsize=11)
ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print("\n✓ XGBoost is SMOOTHER (ensemble averaging)")
print("✗ But still has micro-steps and can't guarantee smoothness")
print("✗ No awareness of resonance physics")

### 🟡 Critical Insight #2: Ensembles Help, But...

XGBoost averages many trees, which smooths predictions.

**BUT:**
- Still piecewise constant at fine scale
- No guarantee of smoothness
- Can't learn resonance physics (Breit-Wigner shape)
- Poor extrapolation beyond training data

---

## Part 3: The Upgrade - Physics-Aware Features

What if we give XGBoost *better features*?

Instead of naive [Z, A, E, MT_onehot], use physics-derived features from the graph:
- **Q-value**: Reaction energy
- **Threshold**: E_threshold
- **ΔZ, ΔA**: Nuclear topology

This is the bridge to deep learning!

In [ ]:
# Get physics-aware tabular projection from FULL training dataset
df_physics = dataset_full.to_tabular(mode='physics')

print("Physics-Aware Features (trained on full EXFOR):")
print(df_physics.columns.tolist())
print(f"\nDataset shape: {df_physics.shape}")
print(f"\nFirst few rows:")
df_physics.head()

In [ ]:
# ============================================================================
# HYPERPARAMETER OPTIMIZATION FOR XGBOOST (PHYSICS FEATURES)
# ============================================================================

print("="*80)
print("STEP 1: HYPERPARAMETER OPTIMIZATION - XGBoost (Physics Features)")
print("="*80)
print("Finding optimal XGBoost configuration for physics-aware features...")
print("This may take 10-15 minutes on large datasets.\n")

# Initialize temporary model for optimization
xgb_optimizer_physics = XGBoostEvaluator()

# Run hyperparameter optimization
opt_result_xgb_physics = xgb_optimizer_physics.optimize_hyperparameters(
    df_physics,
    max_evals=50,          # Increase to 100+ for production
    cv_folds=3,
    verbose=True
)

print("\n" + "="*80)
print("STEP 2: TRAINING FINAL MODEL WITH OPTIMAL HYPERPARAMETERS")
print("="*80)

# Create model with optimal hyperparameters
xgb_physics = XGBoostEvaluator(**opt_result_xgb_physics['best_params'])

# Train on full dataset
xgb_metrics_physics = xgb_physics.train(df_physics)

print("\n" + "="*60)
print("Optimized XGBoost Performance (Physics Features):")
print("="*60)
for key, value in xgb_metrics_physics.items():
    if value is not None:
        print(f"  {key:20s}: {value}")

print("\n" + "="*60)
print("COMPARISON: Naive vs Physics Features (both optimized)")
print("="*60)
print(f"  Test MSE (Naive):   {xgb_metrics_naive['test_mse']:.4e}")
print(f"  Test MSE (Physics): {xgb_metrics_physics['test_mse']:.4e}")
improvement = (xgb_metrics_naive['test_mse'] - xgb_metrics_physics['test_mse']) / xgb_metrics_naive['test_mse'] * 100
print(f"  Improvement: {improvement:.1f}%")
print("\n💡 Physics features provide measurable improvement, but fundamental issues remain!")

In [ ]:
# ============================================================================
# FIXED ENERGY RANGE FOR FINAL COMPARISON
# ============================================================================
E_MIN_PLOT = 1e-4   # 10^-4 eV
E_MAX_PLOT = 1e7    # 10^7 eV

energy_range = (E_MIN_PLOT, E_MAX_PLOT)

# Get physics-mode predictions for U-235
energies_xgb_phys, predictions_xgb_phys = xgb_physics.predict_resonance_region(
    Z, A, mt_code, energy_range, num_points=1000, mode='physics'
)

# Final comparison plot
fig, ax = plt.subplots(figsize=(14, 7))

# Ground truth - BEHIND (zorder=1)
ax.plot(df_truth['Energy'], df_truth['CrossSection'], 
        'b-', linewidth=2.5, label='Ground Truth (EXFOR)', alpha=0.6, zorder=1)

# XGBoost naive - MIDDLE (zorder=2)
ax.plot(energies_xgb, predictions_xgb, 
        'orange', linewidth=2.0, linestyle='--', label='XGBoost (Naive Features)', alpha=0.7, zorder=2)

# XGBoost physics - ON TOP (zorder=3)
ax.plot(energies_xgb_phys, predictions_xgb_phys, 
        'g-', linewidth=2.5, label='XGBoost (Physics Features)', alpha=0.9, zorder=3)

ax.set_xscale('log')
ax.set_yscale('log')
ax.set_xlim(E_MIN_PLOT, E_MAX_PLOT)
ax.set_xlabel('Energy (eV)', fontsize=13, fontweight='bold')
ax.set_ylabel('Cross Section (barns)', fontsize=13, fontweight='bold')
ax.set_title('Physics Features Help... But We Can Do Better!\nU-235 Fission Resonance Region (Model trained on full EXFOR)',
             fontsize=15, fontweight='bold')
ax.legend(fontsize=12, loc='best')
ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print("\n✓ Physics features improve accuracy")
print("✓ Model learns about thresholds and reaction energetics")
print("✓ Training on full EXFOR allows transfer learning to specific isotopes")
print("✗ STILL can't guarantee smooth resonance curves")
print("✗ STILL poor extrapolation to unseen energy ranges")
print("✗ No explicit physics constraints (unitarity, conservation laws)")

### 🟢 Critical Insight #3: Features Matter, But Architecture Matters More

Physics-aware features help XGBoost understand reactions better.

**BUT** the fundamental problem remains:
- Tree-based models are **piecewise constant**
- No inductive bias for **smoothness**
- No way to encode **physical constraints**

---

## Part 4: Feature Importance Analysis

Let's see what XGBoost "thinks" is important.

In [ ]:
# Get feature importance
importance_physics = xgb_physics.get_feature_importance()

# Plot
fig, ax = plt.subplots(figsize=(10, 6))
ax.barh(importance_physics['Feature'], importance_physics['Importance'])
ax.set_xlabel('Importance (Gain)', fontsize=12, fontweight='bold')
ax.set_title('XGBoost Feature Importance (Physics Mode)', fontsize=14, fontweight='bold')
ax.invert_yaxis()
plt.tight_layout()
plt.show()

print("\nTop 5 Most Important Features:")
print(importance_physics.head())

### 🎓 Key Takeaway

> **Low MSE on test data does NOT guarantee safe reactor predictions!**
>
> We need models that:
> 1. Respect physics (smoothness, thresholds, unitarity)
> 2. Extrapolate correctly (beyond training data)
> 3. Prioritize safety-critical reactions (sensitivity weighting)
> 4. **Handle data-sparse scenarios** (like Cl-35) without overfitting
>
> This is why we need **Physics-Informed Deep Learning**.

---

## Summary: Why Classical ML Fails

| Issue | U-235 (Data-Rich) | Cl-35 (Data-Sparse) |
|-------|-------------------|---------------------|
| Staircase Effect | 🔴 Severe (even with lots of data) | 🔴 Severe |
| Interpolation | 🟡 Approximate | 🔴 Very poor (large gaps) |
| Extrapolation | 🔴 Fails | 🔴 Completely fails |
| Physics Constraints | 🔴 None | 🔴 None |
| Uncertainty Quantification | 🔴 Poor | 🔴 Very poor |
| Training Speed | 🟢 Fast | 🟢 Fast |

### The Path Forward

We need:
1. **Graph Neural Networks** → Learn nuclear topology (not just Z, A)
2. **Transformers** → Learn smooth energy sequences σ(E)
3. **Physics-Informed Loss** → Enforce unitarity, thresholds, conservation
4. **Transfer Learning** → Use U-235 knowledge to improve Cl-35 predictions
5. **Uncertainty Quantification** → Know when to trust sparse-data predictions

---

## Next Steps

In **Notebook 01**, we'll:
- Build the **Chart of Nuclides as a Graph**
- Visualize nuclear topology connecting U-235 and Cl-35
- Understand how GNNs can transfer knowledge between isotopes

In **Notebook 02**, we'll:
- Implement **GNN + Transformer**
- Train on graph-structured real data
- See **smooth, physics-compliant predictions** for both isotopes!

In **Notebook 03**, we'll:
- Integrate with **OpenMC** for U-235 reactor validation
- Achieve reactor-grade accuracy with real nuclear data
- Demonstrate uncertainty quantification for Cl-35

Continue to `01_Data_Fabric_and_Graph.ipynb` →